In [1]:
import pandas as pd
from transformers import (
    MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
)
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import Dataset

In [2]:
df = pd.read_csv("hf://datasets/salehalmansour/english-to-arabic-translate/en_ar_final.tsv", sep="\t")
df = df[['en', 'ar']]
df = df.head(50000)
dataset = Dataset.from_pandas(df)


In [3]:
model_name = "Helsinki-NLP/opus-mt-en-ar"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

In [4]:
lora_config = LoraConfig(
    r=2,           # Rank of the low-rank update matrices
    lora_alpha=16,  # Scaling factor for LoRA updates
    target_modules=["q_proj", "v_proj"],  # LoRA applied to attention layers
    lora_dropout=0.1,  # Dropout for LoRA layers
    bias="none"     # No bias added in LoRA layers
)

model = get_peft_model(model, lora_config)


In [5]:
def tokenize_function(examples):
    inputs = tokenizer(examples['en'], truncation=True, padding="max_length", max_length=128)
    targets = tokenizer(examples['ar'], truncation=True, padding="max_length", max_length=128)
    inputs['labels'] = targets['input_ids']
    return inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)


Parameter 'function'=<function tokenize_function at 0x000002665C102A60> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [10]:
print(tokenized_dataset)

Dataset({
    features: ['en', 'ar', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 50000
})


In [12]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./lora-opus-mt-en-ar",  # Save directory
    evaluation_strategy="steps",        # Evaluate every few steps instead of epochs
    eval_steps=500,                     # Evaluate every 500 steps (adjust based on dataset size)
    learning_rate=1e-3,                 # Increase learning rate for faster convergence
    per_device_train_batch_size=64,     # Increase batch size if GPU memory allows
    per_device_eval_batch_size=16,      # Match the evaluation batch size
    gradient_accumulation_steps=2,      # Accumulate gradients to simulate larger batch size
    weight_decay=0.01,                  # Keep weight decay for regularization
    save_total_limit=1,                 # Reduce saved checkpoints to save disk space
    save_steps=1000,                    # Save checkpoints every 1000 steps
    num_train_epochs=1,                 # Use fewer epochs to reduce total time
    predict_with_generate=True,         # Still generate predictions for validation
    logging_dir="./logs",               # Directory for logs
    logging_steps=100,                  # Log every 100 steps to reduce logging overhead
    report_to="none",                   # Disable logging to external tools (e.g., TensorBoard)
    fp16=True,                          # Use mixed precision for faster training
    
)

c:\Users\BLU-RAY\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,  # Using same dataset for simplicity (can be split)
)

trainer.train()

  0%|          | 0/391 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [18]:
model.save_pretrained("./lora-opus-mt-en-ar")
tokenizer.save_pretrained("./lora-opus-mt-en-ar")

('./lora-opus-mt-en-ar/tokenizer_config.json',
 './lora-opus-mt-en-ar/special_tokens_map.json',
 './lora-opus-mt-en-ar/vocab.json',
 './lora-opus-mt-en-ar/source.spm',
 './lora-opus-mt-en-ar/target.spm',
 './lora-opus-mt-en-ar/added_tokens.json')

In [19]:
!zip -r lora-opus-mt-en-ar.zip ./lora-opus-mt-en-ar
from google.colab import files

files.download("lora-opus-mt-en-ar.zip")

  adding: lora-opus-mt-en-ar/ (stored 0%)
  adding: lora-opus-mt-en-ar/checkpoint-1562/ (stored 0%)
  adding: lora-opus-mt-en-ar/checkpoint-1562/scheduler.pt (deflated 56%)
  adding: lora-opus-mt-en-ar/checkpoint-1562/training_args.bin (deflated 51%)
  adding: lora-opus-mt-en-ar/checkpoint-1562/optimizer.pt (deflated 12%)
  adding: lora-opus-mt-en-ar/checkpoint-1562/adapter_config.json (deflated 53%)
  adding: lora-opus-mt-en-ar/checkpoint-1562/rng_state.pth (deflated 25%)
  adding: lora-opus-mt-en-ar/checkpoint-1562/README.md (deflated 66%)
  adding: lora-opus-mt-en-ar/checkpoint-1562/trainer_state.json (deflated 73%)
  adding: lora-opus-mt-en-ar/checkpoint-1562/adapter_model.safetensors (deflated 9%)
  adding: lora-opus-mt-en-ar/adapter_config.json (deflated 53%)
  adding: lora-opus-mt-en-ar/source.spm (deflated 49%)
  adding: lora-opus-mt-en-ar/README.md (deflated 66%)
  adding: lora-opus-mt-en-ar/tokenizer_config.json (deflated 68%)
  adding: lora-opus-mt-en-ar/target.spm (deflated

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
model_name_or_path = "./lora-opus-mt-en-ar"
tokenizer = MarianTokenizer.from_pretrained(model_name_or_path)
model = MarianMTModel.from_pretrained(model_name_or_path)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [21]:
# Extracting Pickle Data

import pickle
with open('/content/Transcribed_Sentences (1).pkl', 'rb') as f:
    data = pickle.load(f)

english_sentences = [item['text'] for item in data]

print(english_sentences)

[' She hides her grief and joins in the homecoming celebration.', ' Rokshik stayed at the palace of his personal friend, the dictator Fernandez Marcos.', ' She had an uneventful career.', " Let's eat the chocolate tonight.", ' Newport was a county borough.', ' The Heckman model falls into this type.', ' The writing lost grain fleet in the rural southwestern corner of St. Catharines.', ' It features a day and night camera and video relay.', ' More modern versions of the tax no longer require an actual stamp.', ' The classification is based on the timing of transcription that is temporarily regulated.', ' He wore a white sitter to dinner.', ' It forms the largest artificial lake in Maui, Lake Manantali.', " Fox shuffled the show's airtime repeatedly to make room for other shows.", ' Viner has produced short documentary videos about legal history and comparative law.', ' When the player character touches a monster, they die.', ' The Customs and Border Protection offices recently opened in

In [23]:
# Translating

arabic_sentences = english_sentences.copy()
index = 0
for sentence in english_sentences:
  print(index)
  input_ids = tokenizer(sentence, return_tensors="pt").input_ids
  output_ids = model.generate(input_ids)
  translated_sentence = tokenizer.decode(output_ids[0], skip_special_tokens=True)
  arabic_sentences[index] = translated_sentence
  index += 1


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [24]:
# Saving Data

import pandas as pd
df = pd.DataFrame({'English': english_sentences, 'Arabic': arabic_sentences})
df.to_csv('Translated_data.csv', index=False)

In [36]:
print(df.iloc[3])

English     Let's eat the chocolate tonight.
Arabic                        لنأكل الشكاولة
Name: 3, dtype: object


In [ ]:
# # How to call the fine-tuned model
# # Load the original model
# base_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-ar")

# # Load the LoRA adapters
# model = PeftModel.from_pretrained(base_model, "./lora-opus-mt-en-ar")

# # Now the model has both the original weights and the fine-tuned LoRA parameters.
